In [89]:
import pandas as pd

file_path = "./output3.csv"
df = pd.read_csv(file_path)

In [90]:
df.drop(columns=['address'], inplace=True)

In [91]:
apartment_count = 0
house_count = 0
for index, row in df.iterrows():
    # Check the value of the 'apartment_or_house' column
    if row['apartment_or_house'] == "house":
        # If it's 'house', replace it with 1
        df.at[index, 'apartment_or_house'] = 1
        house_count += 1
    elif row['apartment_or_house'] == "apartment":
        # If it's 'apartment', replace it with 0
        df.at[index, 'apartment_or_house'] = 0
        apartment_count += 1
    

print(apartment_count)
print(house_count)

53106
13829


In [92]:
#Need to drop anything that isn't call for rent
df = df[pd.to_numeric(df['price'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['rooms'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['bathrooms'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['sqft'], errors='coerce').notnull()]
df = df[pd.to_numeric(df['apartment_or_house'], errors='coerce').notnull()]

# Convert 'price' column to numeric
df['price'] = pd.to_numeric(df['price'])
df['rooms'] = pd.to_numeric(df['rooms'])
df['bathrooms'] = pd.to_numeric(df['bathrooms'])
df['sqft'] = pd.to_numeric(df['sqft'])
df["apartment_or_house"] = pd.to_numeric(df["apartment_or_house"])

In [93]:
temp_df = df.drop(columns='city')
print(temp_df.info())

counts = df['apartment_or_house'].value_counts()
print(counts)

<class 'pandas.core.frame.DataFrame'>
Index: 66822 entries, 1 to 66935
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               66822 non-null  int64  
 1   rooms               66822 non-null  int64  
 2   bathrooms           66822 non-null  float64
 3   sqft                66822 non-null  float64
 4   apartment_or_house  66822 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 3.1 MB
None
apartment_or_house
0    52995
1    13827
Name: count, dtype: int64


In [94]:
df = df.drop(columns=['city'])

# Calculate Pearson correlation coefficients
correlation_matrix = df.corr().abs()

# Extract correlations with the target variable
feature_target_correlations = correlation_matrix['price'].drop('price')

# Print feature-target correlations
print("Feature-Target Correlations:")
print(feature_target_correlations)

Feature-Target Correlations:
rooms                 0.373046
bathrooms             0.419781
sqft                  0.185978
apartment_or_house    0.531489
Name: price, dtype: float64


In [22]:
from sklearn.preprocessing import LabelEncoder


# for col in ['city', 'apartment_or_house']:
#     le = LabelEncoder()
#     df[col] = le.fit_transform(df[col].astype(str))
#     label_encoders[col] = le
#     print(label_encoders)



print(df['city'].unique())



NameError: name 'data' is not defined

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import pandas as pd



# Handle missing values if any
imputer = SimpleImputer(strategy='mean')
df['sqft'] = imputer.fit_transform(df['sqft'].values.reshape(-1, 1))

# Split data into train and test sets
X = df.drop(columns=['price'])  # Features
print(X.info())
y = df['price']  # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Gaussian NB model
model = GaussianNB()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

<class 'pandas.core.frame.DataFrame'>
Index: 52995 entries, 0 to 53105
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city                52995 non-null  int32  
 1   rooms               52995 non-null  int64  
 2   bathrooms           52995 non-null  float64
 3   sqft                52995 non-null  float64
 4   apartment_or_house  52995 non-null  int32  
dtypes: float64(2), int32(2), int64(1)
memory usage: 2.0 MB
None
Accuracy: 0.04066421360505708


In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# Split features and target variable
X = df[['city', 'rooms', 'bathrooms', 'sqft', 'apartment_or_house']]
y = df['price']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Epoch 1/50
1339/1339 [==============================] - 5s 3ms/step - loss: 2425617.5000 - val_loss: 1631576.7500
Epoch 2/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1600854.8750 - val_loss: 1589783.3750
Epoch 3/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1576215.7500 - val_loss: 1578907.7500
Epoch 4/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1567890.3750 - val_loss: 1572773.6250
Epoch 5/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1563805.2500 - val_loss: 1571341.2500
Epoch 6/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1561428.6250 - val_loss: 1569752.1250
Epoch 7/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1559805.2500 - val_loss: 1570029.5000
Epoch 8/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1559265.7500 - val_loss: 1568459.8750
Epoch 9/50
1339/1339 [==============================] - 4s 3ms/step - loss: 1558110.7500